In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from datetime import datetime
from train import *
from loss import *
from accuracy import *
from model import *

import cifar_input
import os.path
import time
import numpy as np
import tensorflow as tf

In [2]:
max_steps = 20000
train_dir = 'cifar10_resnet_model/'
batch_size = 128
log_device_placement = False

In [3]:
with tf.Graph().as_default():
    global_step = tf.contrib.framework.get_or_create_global_step()
    
    # Generating images and its labels
    # build_inputs('cifar10/cifar100', cifar dataset dir, batch size, mode)
    images, labels = cifar_input.build_input('cifar10', '../../cifar/cifar10/data_batch*', batch_size, 'train')
    
    # Creating graph. NUM_CLASESS=10 (CIFAR-10) or NUM_CLASESS=100 (CIFAR-100)
    logits = inference(images, NUM_CLASSES=10)
    
    # Loss/Error and Accuracy
    losses = loss(logits, labels)
    accuracies = accuracy(logits, labels)
    
    # Our train_op (Only minimizing loss)
    train_op = train(losses, global_step, batch_size)
    
    
    # SessionRunHook. Logging will be done each x steps.
    class _LoggerHook(tf.train.SessionRunHook):
        
      def begin(self):
        self._step = -1
        # Creating train_dir if it does not exist and writing to log file
        if not os.path.exists(train_dir):
            os.makedirs(train_dir)
        open(train_dir+'training_data.csv', 'w').close()
        f = open(train_dir+"log.txt",'ab')
        f.write('\n\n==== Run ===\nInfo: VGG\n')
        f.close()

      def before_run(self, run_context):
        # Increment step, reset start_time, and asking for loss and accuracy tensor
        self._step += 1
        self._start_time = time.time()
        return tf.train.SessionRunArgs([losses, accuracies])

      def after_run(self, run_context, run_values):
        duration = time.time() - self._start_time # Calculating time
        loss_value = run_values.results[0]
        accuracy_value = run_values.results[1]
        # Printing log, accuracy, and loss
        if self._step % 1 == 0:
          print("{0}: step {1}, error = {2:.4f}, accuracy = {3:.4f}. ({4:.3f} sec/step)\n".format(
              datetime.now(), self._step, loss_value, accuracy_value, float(duration)))
                
          f = open(train_dir+"log.txt",'ab')
          f.write("{0}: step {1}, error = {2:.4f}, accuracy = {3:.4f}. ({4:.3f} sec/step)\n".format(
              datetime.now(), self._step, loss_value, accuracy_value, float(duration)))
          f.close()
                
          f = open(train_dir+"training_data.csv",'ab')
          f.write('{0},{1},{2}\n'.format(self._step, loss_value, accuracy_value))
          f.close()
            
    with tf.train.MonitoredTrainingSession(checkpoint_dir=train_dir,
                                           hooks=[tf.train.StopAtStepHook(last_step=max_steps),
                                                  tf.train.NanTensorHook(losses),
                                                  _LoggerHook()],save_checkpoint_secs=30, 
                                           config=tf.ConfigProto(
                                               log_device_placement=log_device_placement)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(train_op)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into cifar10_resnet_model/model.ckpt.
2017-03-13 13:39:55.491961: step 0, error = 2.3862, accuracy = 0.1094. (11.473 sec/step)

2017-03-13 13:39:59.985166: step 1, error = 2.3310, accuracy = 0.1719. (4.492 sec/step)

2017-03-13 13:40:04.161443: step 2, error = 2.3466, accuracy = 0.1016. (4.175 sec/step)

2017-03-13 13:40:08.333783: step 3, error = 2.3178, accuracy = 0.1484. (4.172 sec/step)

2017-03-13 13:40:12.509073: step 4, error = 2.3937, accuracy = 0.1172. (4.174 sec/step)

2017-03-13 13:40:16.709719: step 5, error = 2.3571, accuracy = 0.1328. (4.200 sec/step)

2017-03-13 13:40:20.897959: step 6, error = 2.3390, accuracy = 0.1094. (4.188 sec/step)

INFO:tensorflow:Saving checkpoints for 8 into cifar10_resnet_model/model.ckpt.
2017-03-13 13:40:26.415020: step 7, error = 2.3331, accuracy = 0.1484. (5.516 sec/step)

2017-03-13 13:40:30.618605: step 8, error = 2.3381, accuracy = 0.1562. (4.203 sec/ste

KeyboardInterrupt: 